In [86]:
import pytest
import tensorflow as tf
import pandas as pd
import numpy as np
from scripts.utils.utils.utils import (
    ZeroBond,
    FinanceUtils
)
from scripts.trainer.trainer import simulate
# Params for check
T = 8
nsteps = 1000
# Fixed params
simple_params = {
    "T": T,
    "N_steps": nsteps,
    "dim": 1,
    "sigma": 0.01,
    "nsims": 1
}
data, _ = simulate(
    T = simple_params["T"], 
    N_steps = simple_params["N_steps"], 
    dim = simple_params["dim"],
    sigma = simple_params["sigma"], 
    nsims = simple_params["nsims"]
)

# Get data to process the ZeroBound
delta_x = data.delta_x_0.values
xt = data.X_0.values
dt = data.dt.values
t_unique = data.dt.unique()
dict_C = {dt:FinanceUtils.C(dt, sigma_value = 0.01) for dt in t_unique}
ct = data.apply(lambda x: dict_C[x['dt']], axis = 1)
nt = ZeroBond.N_tensor(dt,xt,ct)
# Convert to tensors
xt = tf.convert_to_tensor(xt, dtype = tf.float64)
delta_x = tf.convert_to_tensor(delta_x,dtype = tf.float64)
dt = tf.convert_to_tensor(dt, dtype = tf.float64)
ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
T = tf.Variable(np.float64(T), name = 'T', trainable=False)
# Fix the batch size
batch_size = int(xt.shape[0] / nsteps)
# Real values
v_real = ZeroBond.Z_tensor(xt, dt, T, ct)
v_real_reshaped = tf.reshape(v_real,(batch_size,nsteps))
n_tensor = ZeroBond.N_tensor(dt, xt, ct)
# Derivative:
xt = tf.Variable(xt, name = 'xn', trainable = True)
dt = tf.Variable(dt, name = 'tn', trainable = False)
ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
with tf.GradientTape() as tape:
    y = ZeroBond.Z_normalized(xt, dt, T, ct)
grad_df = tape.gradient(y, {'xn':xt})
grads = grad_df['xn']
# Simulate - LGM step:
grads_reshaped = tf.reshape(grads, (batch_size, nsteps))
xt_reshaped = tf.reshape(xt, (batch_size, nsteps))
delta_x_reshaped = tf.reshape(delta_x, (batch_size, nsteps))
# Calculate the MVP
v = np.ones((batch_size, nsteps)) * np.float64(v_real_reshaped[0, 0])
for i in range(1, nsteps):
    v[:, i] = (v[:, i - 1] + grads_reshaped[:, i - 1] * delta_x_reshaped[:, i])
# Calculate errors absolute
v_real = np.array(tf.reshape(v_real_reshaped[0, :], -1))
v_column = np.array(tf.reshape(v[0, :], -1)) * n_tensor.numpy()
dt_list = np.array(dt)
df_results = pd.DataFrame(zip(xt.numpy(), v_real, v_column, dt_list, n_tensor.numpy()), columns = ["xt", "v_real","v_est","dt", "n"])
# Error
df_results["absolute_error"] = (df_results.v_real - df_results.v_est).abs()

In [87]:
df_results

,xt,v_real,v_est,dt,n,absolute_error
0,0.000000,0.786628,0.786628,0.000000,1.000000,0.000000
1,-0.001371,0.794807,0.794783,0.008008,1.000229,0.000023
2,-0.002902,0.804002,0.803945,0.016016,1.000434,0.000057
3,-0.002861,0.803914,0.803875,0.024024,1.000652,0.000039
4,-0.003718,0.809161,0.809123,0.032032,1.000842,0.000038
...,...,...,...,...,...,...
995,-0.045540,1.000097,1.004859,7.967968,0.931134,0.004762
996,-0.043680,1.000034,1.004795,7.975976,0.943992,0.004761
997,-0.044492,1.000034,1.004779,7.983984,0.938372,0.004745
998,-0.044664,1.000018,1.004786,7.991992,0.937192,0.004768


In [39]:
df_x

,dt,sim,X_0,_delta_x_0,delta_x_0
0,0.000000,0,0.000000,NaN,0.000000
1,0.080808,0,0.005703,0.000000,0.005703
2,0.161616,0,0.004179,0.005703,-0.001524
3,0.242424,0,0.004540,0.004179,0.000361
4,0.323232,0,0.008219,0.004540,0.003680
...,...,...,...,...,...
995,7.676768,9,0.039795,0.037477,0.002318
996,7.757576,9,0.035808,0.039795,-0.003987
997,7.838384,9,0.044382,0.035808,0.008574
998,7.919192,9,0.040467,0.044382,-0.003915
